In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
file_path='/content/drive/MyDrive/Kidney_segmentation_data/'

Mounted at /content/drive/


In [ ]:
import os
from glob import glob
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
import numpy as np
from sklearn.decomposition import PCA, IncrementalPCA
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import albumentations as A
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dense
#from keras.layers import Flatten
from keras.layers import AveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Lambda
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
from sklearn.preprocessing import StandardScaler
import math


In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from tensorflow.keras.metrics import Recall, Precision
from keras.optimizers import Adam
%matplotlib inline 

In [ ]:
# Enable mixed precision and tensor float 32 execution
tf.config.experimental.enable_mixed_precision = True
tf.config.experimental.enable_tensor_float_32_execution = True
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
print(physical_devices)
tf.config.run_functions_eagerly(True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
def Augmentation(images,mask):
    def _transform(image,mask):
        transform=A.Compose([
         A.HorizontalFlip(p=0.25),
         A.VerticalFlip(p=0.25),
         A.RandomRotate90(p=.5),
         A.RandomContrast(limit=0.2, p=0.1),
         A.Transpose(p=0.5),
         A.ElasticTransform(p=.4, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    ])
        aug=transform(image=image,mask=mask)
        img=aug['image']
        img=tf.cast(img,tf.float64)
        msk=aug['mask']
        msk=tf.cast(msk,tf.float64)
#         print("after augmentation",img.shape,msk.shape)
        return img,msk
    image,mask=tf.numpy_function(_transform,[images,mask],[tf.float64,tf.float64])
    image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    mask.set_shape([IMAGE_SIZE, IMAGE_SIZE, 2])
    return image,mask

In [ ]:
IMAGE_SIZE=256
AUTO = tf.data.experimental.AUTOTUNE
def load_data(split=0.3):
    images = sorted(glob(file_path + "batch_256/*jpg"), key=lambda x: int(x.split("/")[-1].split(".")[0]))
    masks = sorted(glob(file_path+"masks_256/*"))
    total_size = len(images)
    valid_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=split, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split, random_state=42)

    valid_x, test_x = train_test_split(valid_x, test_size=split, random_state=42)
    valid_y, test_y = train_test_split(valid_y, test_size=split, random_state=42)

    return (train_x, train_y), (valid_x, valid_y),(test_x,test_y)
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)=load_data()
train_size=len(train_x)
valid_size=len(valid_x)



def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x,(256,256),interpolation=cv2.INTER_AREA)
    x = x/255.0
    x=tf.cast(x,dtype=tf.float64)
    return x
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(256,256),interpolation=cv2.INTER_NEAREST)
    x = np.expand_dims(x, axis=-1)
    x = np.concatenate([x,x],axis=-1)
    x=tf.cast(x,dtype=tf.float64)
    return x

def parser(x,y):
    def _parse(x,y):
        x=read_image(x)
        y=read_mask(y)
        return x,y
    x,y = tf.numpy_function(_parse, [x,y], [tf.float64,tf.float64])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 2])
    return x,y
    
def tf_dataset(x, y, batch,repeat_original=1):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(parser)
    original_dataset = dataset.repeat(repeat_original)

    augmented_dataset = dataset.map(Augmentation)
    dataset = tf.data.Dataset.concatenate(original_dataset, augmented_dataset)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(128,seed=0)
    dataset = dataset.batch(batch)
    return dataset
train_dataset=tf_dataset(train_x,train_y,batch=8)
valid_dataset=tf_dataset(valid_x,valid_y,batch=8)

IMAGE_WIDTH=256
IMAGE_HEIGHT=256
IMAGE_CHANNELS=3

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [ ]:
#LOSS FUNCTIONS 
from tensorflow.keras.losses import binary_crossentropy
smooth = 1
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def binary_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred)

def binary_dice_loss(y_true, y_pred):
    d_loss= dice_loss(y_true, y_pred)
    b_loss= binary_loss(y_true, y_pred)
    loss=(d_loss+b_loss)
    return loss

def Tversky_Index(y_true,y_pred,smooth):
    y_true= tf.keras.layers.Flatten()(y_true)
    y_pred= tf.keras.layers.Flatten()(y_pred)
    true_positive= tf.reduce_sum(y_true*y_pred)
    false_negative=tf.reduce_sum(y_true*(1-y_pred))
    false_positive=tf.reduce_sum((1-y_true)*y_pred)
    alpha=.7
    TI=(true_positive+smooth)/(true_positive+alpha*false_negative+(1-alpha)*false_positive+smooth)
    return TI

def Tversky_loss(y_true,y_pred):
    return 1-Tversky_Index(y_true,y_pred,smooth=1)
def focal_Tversky_loss(y_true,y_pred):
    pt_1 = Tversky_Index(y_true, y_pred,smooth=1)
    gamma = 0.75
    return tf.math.pow((1-pt_1), gamma)

In [ ]:
from keras.applications import VGG19
from keras.layers import GlobalAveragePooling2D 
from keras.layers import multiply, Reshape
from tensorflow.keras.optimizers import Nadam ,Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications import *

In [ ]:
#models
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = multiply([init, se])
    return x

def conv_block(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = [inputs]

    model = tf.keras.applications.DenseNet169(include_top=False, weights='imagenet',input_tensor=inputs)
    for layers in model.layers:
        layers.trainable=False
        
    #model.summary()
    names = ["conv1/relu", "pool2_conv", "pool3_conv", "pool4_conv"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("relu").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [ 256, 128, 64,64,32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_connections[i]])
        x = conv_block(x, f)

    return x
def encoder2(inputs):
    num_filters = [32,64,64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPooling2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 64,32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x


def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)
def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = concatenate([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    outputs = concatenate([outputs1, outputs2])
    model = Model(inputs, outputs)
    return model


In [ ]:
import tensorflow as tf
model=tf.keras.models.load_model(file_path+"batch/"+"tenth_model.hd5",custom_objects={
        'dice_loss': dice_loss ,'dice_coef': dice_coef})

In [ ]:
# model=build_model((256,256,3))

device = '/GPU:0'
save_filepath= file_path+"batch/"+"eleventh_model.hd5"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(save_filepath,verbose=1,monitor='val_dice_coef',mode='max',save_best_only=True),
    CSVLogger(file_path+"batch/"+"dataDensenet161freeze.csv")
    ]
with tf.device(device):
    model.compile(loss=dice_loss,optimizer=Adam(learning_rate=0.00025),metrics=[dice_coef,Recall(),Precision()])
    t_steps=train_size//8
    v_steps=valid_size//8
with tf.device(device):
    history=model.fit(train_dataset, 
                            epochs =20,
                            steps_per_epoch = t_steps,
                            validation_data = valid_dataset,
                            validation_steps = v_steps,
                            callbacks=callbacks)

Epoch 1/20
822/838 [============================>.] - ETA: 3:04 - loss: 0.1626 - dice_coef: 0.8374 - recall: 0.8492 - precision: 0.9178

/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


838/838 [==============================] - ETA: 0s - loss: 0.1622 - dice_coef: 0.8378 - recall: 0.8486 - precision: 0.9181 
Epoch 1: val_dice_coef improved from -inf to 0.79796, saving model to /content/drive/MyDrive/Kidney_segmentation_data/batch/eleventh_model.hd5


838/838 [==============================] - 12822s 15s/step - loss: 0.1622 - dice_coef: 0.8378 - recall: 0.8486 - precision: 0.9181 - val_loss: 0.2020 - val_dice_coef: 0.7980 - val_recall: 0.8329 - val_precision: 0.9001
Epoch 2/20


/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


...

/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


838/838 [==============================] - ETA: 0s - loss: 0.1616 - dice_coef: 0.8384 - recall: 0.8534 - precision: 0.9124
Epoch 3: val_dice_coef improved from 0.81632 to 0.82457, saving model to /content/drive/MyDrive/Kidney_segmentation_data/batch/eleventh_model.hd5


838/838 [==============================] - 1571s 2s/step - loss: 0.1616 - dice_coef: 0.8384 - recall: 0.8534 - precision: 0.9124 - val_loss: 0.1754 - val_dice_coef: 0.8246 - val_recall: 0.8417 - val_precision: 0.8948
Epoch 4/20


/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


838/838 [==============================] - ETA: 0s - loss: 0.2227 - dice_coef: 0.7773 - recall: 0.7915 - precision: 0.8731
Epoch 4: val_dice_coef did not improve from 0.82457
838/838 [==============================] - 1579s 2s/step - loss: 0.2227 - dice_coef: 0.7773 - recall: 0.7915 - precision: 0.8731 - val_loss: 0.1934 - val_dice_coef: 0.8066 - val_recall: 0.8142 - val_precision: 0.9201
Epoch 5/20
255/838 [========>.....................] - ETA: 14:47 - loss: 0.1627 - dice_coef: 0.8373 - recall: 0.8520 - precision: 0.9181